https://www.axiomoptics.com/wp-content/uploads/2021/09/Alvium_SWIR-Flyer_EN.pdf

In [4]:
import numpy as np
from units import *

In [6]:
# camera array size (Alvium 1800 U-030)
pix = 5*um
print("horiz x vertical:",pix*656/mm, "x", pix*520/mm, "(mm)")

horiz x vertical: 3.28 x 2.5999999999999996 (mm)


## top hat

In [38]:
# this is the size of a diffraction limited beam, not the smallest top hat width
efl = 250*mm
lmbda = 1529*nm
d = 25*mm # input beam dia - check this in CAD
DL = 4*efl*lmbda/(np.pi*d) # output spot full width
DL/um # diffraction limited beam size

19.46783263900064

In [39]:
# top hat quality factor
TH = 100*um
TH/DL

5.136678635692924